In [1]:
import os
import json
dataset_root = '../MATH/train/'

with open(os.path.join(dataset_root, 'train_dataset_eval.jsonl'), 'r') as f:
    lines = f.readlines()

data = [json.loads(line) for line in lines]

print(data[0].keys())
print(len(data))

dict_keys(['prompt', 'predict', 'label'])
7500


In [2]:
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE")
)

In [3]:
import re

def compare_boxed_contents(str1, str2):
    # Regular expression to match the pattern \boxed{content}
    pattern = r'\\boxed\{(.*?)\}'
    
    # Find all occurrences of \boxed{content} in both strings
    match1 = re.search(pattern, str1)
    match2 = re.search(pattern, str2)
    
    # Check if both strings contain the \boxed{} pattern and compare their contents
    if match1 and match2:
        return match1.group(1) == match2.group(1)
    return False  # Return False if either string does not contain a \boxed{}


In [4]:
def tweak(sample, client):
    question = sample['prompt']
    student_answer = sample['predict']
    ground_truth = sample['label']
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant specializing in generating well structured explanations for educational content."},
                {
                    "role": "user",
                    "content": f"""You are a teacher model tasked with improving a student's mathematical answer. The goal is to help the student learn and correct their mistakes, not by simply providing the correct solution, but by modifying the student's answer to make it more accurate. You will receive the following inputs:
Question: The original math problem the student was asked to solve.
Student's Answer: The answer the student has provided for the question.
Ground Truth Answer: The correct, intended solution to the problem.

Your task is to output a tweaked answer that follows the general idea of the student, but correct the mistakes along the way. Your tweaked answer should look like the student has learned from the mistakes and paying attention to what the teacher is saying.

Some requirements you need to meet:
1. Do not make the answet too long! The student should learn to generate a concise and correct answer!
2. Use latex to show the expressions, but do not format the text using \\text{{}} or \\textbf{{}} commands.
3. Use single dollar sign instead of double $$ sign to wrap the latex expressions. This inline math mode is more suitable for the student's level.
4. Use the \\boxed{{}} command to highlight the final answer! Use the \\boxed{{}} command to highlight the final answer!

Question: {question}\nStudent answer: {student_answer}\nGround truth: {ground_truth}\nYour tweaked answer is:"""
                }
            ]
        )
        tweaked_answer = response.choices[0].message.content
        sample['tweak'] = tweaked_answer
        
    except Exception as e:
        pass
        
    return sample

In [5]:
train_tweak_path = os.path.join(dataset_root, 'train_tweak.jsonl')
for ix, sample in tqdm(enumerate(data[5897:]), total=len(data[5897:])):
    if not compare_boxed_contents(sample['predict'], sample['label']):
        sample = tweak(sample, client)
    with open(train_tweak_path, 'a') as f:
        f.write(json.dumps(sample) + '\n')

100%|██████████| 1603/1603 [1:25:42<00:00,  3.21s/it]


In [6]:
with open(train_tweak_path, 'r') as f:
    lines = f.readlines()

data = [json.loads(line) for line in lines]
print(len(data))

7509
